In [5]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.linalg import sqrtm
from scipy.linalg import block_diag

module_path = os.path.abspath(os.path.join('../../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path = os.path.abspath(os.path.join('../../data'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from dynamics import Dynamics
from utils import get_form_initial_conditions
from centralized_newton import CentralizedNewton

In [6]:
# Position estimation error
def position_estimation_error(X_est, X_true):
    return np.linalg.norm(X_est[:3, :, :] - X_true[:3, :, :], axis=0).flatten(), np.linalg.norm(X_est[6:9, :, :] - X_true[6:9, :, :], axis=0).flatten(), np.linalg.norm(X_est[12:15, :, :] - X_true[12:15, :, :], axis=0).flatten(), np.linalg.norm(X_est[18:21, :, :] - X_true[18:21, :, :], axis=0).flatten()

In [7]:
# Simulation parameters
dt = 60.0  # Time step [s]
K = 395  # Duration [min]
M = 1  # Number of Monte-Carlo simulations
N = 4  # Number of satellites
formation = 1
n_x = 6 
n_p = 3  
n = N * n_x
o = 3 + 3 + 2 + 1 
H = 15 # Window size [min]
invalid_rmse = 1e20 # [m]
grad_norm_order_mag = True
grad_norm_tol = 1e-6
max_iterations = 20
dyn = Dynamics()

In [29]:
# Observation noise
r_chief_pos = 1e-1  # [m]
R_chief = np.diag(np.concatenate([r_chief_pos * np.ones(3)])) ** 2
r_deputy_pos = 1e0  # [m]
R_deputies = np.diag(np.concatenate([r_deputy_pos * np.ones(6)])) ** 2
R = block_diag(R_chief, R_deputies)

# Initial deviation noise
p_pos_initial = 1e-2  # [m]
p_vel_initial = 1e-2  # [m / s]
P_0 = np.diag(np.concatenate([p_pos_initial * np.ones(3), p_vel_initial * np.ones(3)])) ** 2
P_0 = block_diag(P_0, P_0, P_0, P_0)

In [30]:
# Seed for reproducib_ility
# np.random.seed(42)

# Initial conditions for the state vector and true state vectors
X_initial = get_form_initial_conditions(formation)
with open(f"../../data/tudatpy_form{formation}_ts_{int(dt)}.pkl", "rb") as file:
    X_true = pickle.load(file)[:, :, :K]

# Algorithm class to use
alg = CentralizedNewton(H, K, o, R, grad_norm_order_mag, grad_norm_tol, max_iterations)

# Observations
Y = np.zeros((o, 1, K))
for k in range(K):
    Y[:, :, k] = alg.h(X_true[:, :, k]) + np.random.multivariate_normal(np.zeros(o), R).reshape((o, 1))

In [47]:
k = 14
x_init = X_initial + np.random.multivariate_normal(np.zeros(n), P_0).reshape((n, 1))
HJ_x = alg.HJ(k, dt, Y, x_init)
T_matrix_x = np.zeros_like(HJ_x)
R_matrix_x = np.zeros_like(HJ_x)
for i in range(N):
    for j in range(N):
        block = HJ_x[i * n_x : (i + 1) * n_x, j * n_x : (j + 1) * n_x]
        if i == j or i == 0 or j == 0:
            T_matrix_x[i * n_x : (i + 1) * n_x, j * n_x : (j + 1) * n_x] = block
        else:
            R_matrix_x[i * n_x : (i + 1) * n_x, j * n_x : (j + 1) * n_x] = -block
assert min(np.linalg.eigvalsh(T_matrix_x)) > 0 and np.allclose(T_matrix_x, T_matrix_x.T)

In [87]:
import numpy as np
from scipy.linalg import sqrtm

def construct_L_i(i, A_i, B_i):
    n_blocks = N
    block_size = n_x
    total_size = n_blocks * block_size
    L_i = np.eye(total_size)

    # Define index ranges
    i_start = i * block_size
    i_end = (i + 1) * block_size

    # Compute L_i
    sqrt_A_i = sqrtm(A_i)
    L_i[i_start:i_end, i_start:i_end] = sqrt_A_i
    L_i[i_end:, i_start:i_end] = B_i @ np.linalg.inv(sqrt_A_i)

    return L_i

def cholesky_custom(A):
    n_blocks = N
    block_size = n_x
    total_size = n_blocks * block_size
    A_current = A.copy()
    
    L_list = []
    for i in range(n_blocks):
        i_start = i * block_size
        i_end = (i + 1) * block_size

        A_i = A_current[i_start:i_end, i_start:i_end]
        B_i = A_current[i_end:, i_start:i_end] 
        C_i = A_current[i_end:, i_end:] 

        L_i = construct_L_i(i, A_i, B_i)
        L_list.append(L_i)

        # Refresh A (1)
        # L_i_inv = np.linalg.inv(L_i)
        # A_current = L_i_inv @ A_current @ L_i_inv.T
        
        # Refresh A (2)
        A_current = np.eye(total_size)
        A_current[i_end:, i_end:] = C_i - B_i @ np.linalg.inv(A_i) @ B_i.T

    # Final L = L_1 L_2 ... L_n
    L = L_list[0]
    for L_i in L_list[1:]:
        L = L @ L_i

    return L

In [88]:
# L = np.linalg.cholesky(T_matrix_x)
L_custom = cholesky_custom(T_matrix_x)
np.allclose(L_custom @ L_custom.T, T_matrix_x)

True

In [ ]:
def construct_Q_i(i, A_i, B_i):
    n_blocks = N
    block_size = n_x
    total_size = n_blocks * block_size
    L_i = np.eye(total_size)

    # Index ranges
    i_start = i * block_size
    i_end = (i + 1) * block_size

    # Set sqrt_aii at (i, i)
    L_i[i_start:i_end, i_start:i_end] = sqrtm(A_i)

    # Set bi @ inv_sqrt_aii at (j, i) blocks
    bi_blocks = B_i.shape[0] // block_size
    for j in range(bi_blocks):
        j_start = (i + 1 + j) * block_size
        j_end = j_start + block_size
        L_i[j_start:j_end, i_start:i_end] = B_i[j*block_size:(j+1)*block_size] @ np.linalg.inv(sqrtm(A_i))

    return L_i

def cholesky_custom(A):
    n_blocks = N
    block_size = n_x
    A_current = A.copy()
    
    L_list = []
    for i in range(n_blocks):
        i_start = i * block_size
        i_end = (i + 1) * block_size

        A_i = A_current[i_start:i_end, i_start:i_end]
        B_i = A_current[i_end:, i_start:i_end]
        C_i = A_current[i_end:, i_end:]

        Q_i = construct_Q_i(i, A_i, B_i)
        L_list.append(Q_i)

        # A_current = np.linalg.inv(L_i) @ A_current @ np.linalg.inv(L_i).T
        A_current[i_start:i_end, i_start:i_end] = np.eye(block_size)
        A_current[i_start:i_end, i_end:] = 0
        A_current[i_end:, i_start:i_end] = 0
        A_current[i_end:, i_end:] = C_i - B_i @ np.linalg.inv(A_i) @ B_i.T

    # Multiply all L_i to get final L
    L = L_list[0]
    for L_i in L_list[1:]:
        L = L @ L_i

    return L